In [1]:
import torch
import torch.nn as nn
import torchtext
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import time
gpu = ('cuda:0')

In [13]:
test_dataset = {
    'data' : [
        "1 2 3 4 5 6 7 8 9 10",
        "1 2 3 4 5 6 7 8 9",
        "1 2 3 4 5 6 7 8",
        "1 2 3 4 5 6 7",
        "1 2 3 4 5 6",
        "1 2 3 4 5",
        "1 2 3 4",
        "1 2 3",
        "1 2",
        "1"
        ],
        
    'label' : [
        'pos',
        'neg',
        'pos',
        'neg',
        'pos',
        'neg',
        'pos',
        'neg',
        'pos',
        'neg'
    ]
        }

In [15]:
import pandas as pd
dataset = pd.DataFrame(test_dataset)

In [16]:
dataset.head()

,data,label
0,1 2 3 4 5 6 7 8 9 10,pos
1,1 2 3 4 5 6 7 8 9,neg
2,1 2 3 4 5 6 7 8,pos
3,1 2 3 4 5 6 7,neg
4,1 2 3 4 5 6,pos


In [28]:
dataset['sentiment'] = dataset['label'].map({'neg':0, 'pos':1}) 
dataset = dataset.iloc[:,::2]

In [29]:
dataset.head()

,data,sentiment
0,1 2 3 4 5 6 7 8 9 10,1
1,1 2 3 4 5 6 7 8 9,0
2,1 2 3 4 5 6 7 8,1
3,1 2 3 4 5 6 7,0
4,1 2 3 4 5 6,1


In [30]:
dataset.to_csv('dataset.csv', index=False)

In [2]:
TEXT = torchtext.data.Field(
    sequential = True,
    lower = True,
    batch_first = True,
    tokenize = 'spacy',
    pad_first = True
)

LABEL = torchtext.data.Field(
    sequential = False,
    use_vocab = False,
    is_target = True
)

In [3]:
dataset_ = torchtext.data.TabularDataset(
    path = 'dataset.csv', 
    format = 'csv',
    skip_header = True,
    fields = [('data', TEXT), ('sentiment', LABEL)] 
)

In [4]:
print(dataset_[0].data)
print(dataset_[0].sentiment)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
1


In [46]:
# why always shuffling ?
train_dataset, validation_dataset = dataset_.split(0.6)

In [22]:
len(train_dataset)

6

In [47]:
train_dataset[0].data

['1', '2', '3', '4']

In [24]:
TEXT.build_vocab(train_dataset)
vocab = TEXT.vocab
len(vocab)
# vocab.stoi

11

In [25]:
train_iter, valid_iter = torchtext.data.BucketIterator.splits(
    (train_dataset, validation_dataset), 
    sort_key = lambda x: len(x.data),
    batch_sizes = (2, 2),
    device = gpu
)

In [26]:
# for accessing specific batch
every_batch = []
for batch in enumerate(train_iter):
    every_batch.append(batch)
    # print(batch)
    # inputs, labels = batch.Review, batch.Label
    # print(inputs.shape, labels.shape)

In [32]:
a,b = every_batch[2][1]

In [33]:
a

tensor([[2, 3, 4, 5, 6],
        [1, 1, 1, 2, 3]], device='cuda:0')